# Chapters with only "frequent" words

Task: find the chapters without more than 20 rare words, where a rare word has a frequency (as lexeme) of less than 70.

A question posed by Oliver Glanz.

<img align="right" src="images/tf-small.png" width="128"/>
<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/dans-small.png"/>

# Sets and queries

You can pass custom sets to the search function, as we have seen in [advanced](searchAdvanced.ipynb).
Now we want to give a real-world example of that, and also show how you can prepare sets for use
in the TF browser.

## Chapters with only "frequent" words

The following task comes from the department of education:

*Find the chapters without more than 20 rare words, where a rare word has a frequency (as lexeme) of less than 70.*

A question posed by Oliver Glanz.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

In [3]:
from tf.fabric import Fabric
from tf.app import use
from tf.lib import writeSets, readSets

By way of variation, we do not use the *minimal incantation* but a *fast* incantation.

In [4]:
TF = Fabric(modules='etcbc/bhsa/tf/c')

This is Text-Fabric 7.0.3
Api reference : https://dans-labs.github.io/text-fabric/Api/General/
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

114 features found and 0 ignored


In [5]:
api = TF.load('book freq_lex', silent=True)

In [6]:
A = use('bhsa', api=api, hoist=globals())

**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="provenance of BHSA = Biblia Hebraica Stuttgartensia Amstelodamensis">BHSA</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Writing/Hebrew" title="('Hebrew characters and transcriptions',)">Character table</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/c/0_home.html" title="BHSA feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Bhsa/" title="bhsa API documentation">bhsa API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 7.0.3</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Search/#search-templates" title="Search Templates Introduction and Reference">Search Reference</a>

In [7]:
FREQ = 70
AMOUNT = 20

## Query

A straightforward query is:

In [8]:
query = f'''
chapter
/without/
  word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
  < word freq_lex<{FREQ}
/-/
'''

Two problems with this query:

* it is very inelegant
* it does not perform, in fact, you cannot wait for it.

So, better not search with this one.

In [9]:
# indent(reset=True)
# info('start query')
# results = S.search(query, limit=1)
# info('end query')
# len(results)

# By hand

On the other hand, with a bit of hand coding it is very easy, and almost instantaneous:

In [10]:
results = []
allChapters = F.otype.s('chapter')

for chapter in allChapters:
    if len([
        word for word in L.d(chapter, otype='word') if F.freq_lex.v(word) < FREQ
    ]) < AMOUNT:
        results.append(chapter)
        
print(f'{len(results)} chapters out of {len(allChapters)}')

60 chapters out of 929


In [11]:
resultsByBook = dict()

for chapter in results:
    (bk, ch, vs) = T.sectionFromNode(chapter)
    resultsByBook.setdefault(bk, []).append(ch)
    
for (bk, chps) in resultsByBook.items():
    print('{} {}'.format(bk, ', '.join(str(c) for c in chps)))

Exodus 11, 24
Leviticus 17
Deuteronomy 30
Joshua 23
Isaiah 12, 39
Jeremiah 45
Ezekiel 15
Hosea 3
Joel 3
Psalms 1, 3, 4, 13, 14, 15, 20, 23, 24, 26, 43, 47, 53, 54, 61, 67, 70, 82, 86, 87, 93, 97, 99, 100, 101, 110, 113, 114, 115, 117, 120, 121, 122, 123, 124, 125, 126, 127, 128, 130, 131, 133, 134, 136, 138, 150
Job 25
Esther 10
2_Chronicles 27


# Custom sets

Once you have these chapters, you can put them in a set and use them in queries.

We show how to query results as far as they occur in an "ordinary" chapter.

First we search for a phenomenon in all chapters. The phenomenon is a clause with a subject consisting of a single noun in
the plural and a verb in the plural.

In [12]:
sets = dict(ochapter=set(results))

In [13]:
query1 = '''
verse
  clause
    phrase function=Pred
      word pdp=verb nu=sg
    phrase function=Subj
      =: word pdp=subs nu=pl
      :=
'''

In [14]:
results1 = A.search(query1)

  2.07s 263 results


In [15]:
A.table(results1, start=1, end=10)

n | verse | clause | phrase | word | phrase | word
--- | --- | --- | --- | --- | --- | ---
1|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1">Genesis 1:1</a>בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ </span>|<span class="hb">בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ׃ </span>|<span class="hb">בָּרָ֣א </span>|<span class="hb">בָּרָ֣א </span>|<span class="hb">אֱלֹהִ֑ים </span>|<span class="hb">אֱלֹהִ֑ים </span>
2|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3" sec="Genesis 1:3">Genesis 1:3</a>וַיֹּ֥אמֶר אֱלֹהִ֖ים יְהִ֣י אֹ֑ור וַֽיְהִי־אֹֽור׃ </span>|<span class="hb">וַיֹּ֥אמֶר אֱלֹהִ֖ים </span>|<span class="hb">יֹּ֥אמֶר </span>|<span class="hb">יֹּ֥אמֶר </span>|<span class="hb">אֱלֹהִ֖ים </span>|<span class="hb">אֱלֹהִ֖ים </span>
3|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">Genesis 1:4</a>וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃ </span>|<span class="hb">וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור </span>|<span class="hb">יַּ֧רְא </span>|<span class="hb">יַּ֧רְא </span>|<span class="hb">אֱלֹהִ֛ים </span>|<span class="hb">אֱלֹהִ֛ים </span>
4|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4">Genesis 1:4</a>וַיַּ֧רְא אֱלֹהִ֛ים אֶת־הָאֹ֖ור כִּי־טֹ֑וב וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃ </span>|<span class="hb">וַיַּבְדֵּ֣ל אֱלֹהִ֔ים בֵּ֥ין הָאֹ֖ור וּבֵ֥ין הַחֹֽשֶׁךְ׃ </span>|<span class="hb">יַּבְדֵּ֣ל </span>|<span class="hb">יַּבְדֵּ֣ל </span>|<span class="hb">אֱלֹהִ֔ים </span>|<span class="hb">אֱלֹהִ֔ים </span>
5|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=5&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:5" sec="Genesis 1:5">Genesis 1:5</a>וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לָאֹור֙ יֹ֔ום וְלַחֹ֖שֶׁךְ קָ֣רָא לָ֑יְלָה וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר יֹ֥ום אֶחָֽד׃ פ </span>|<span class="hb">וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לָאֹור֙ יֹ֔ום </span>|<span class="hb">יִּקְרָ֨א </span>|<span class="hb">יִּקְרָ֨א </span>|<span class="hb">אֱלֹהִ֤ים׀ </span>|<span class="hb">אֱלֹהִ֤ים׀ </span>
6|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=6&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:6" sec="Genesis 1:6">Genesis 1:6</a>וַיֹּ֣אמֶר אֱלֹהִ֔ים יְהִ֥י רָקִ֖יעַ בְּתֹ֣וךְ הַמָּ֑יִם וִיהִ֣י מַבְדִּ֔יל בֵּ֥ין מַ֖יִם לָמָֽיִם׃ </span>|<span class="hb">וַיֹּ֣אמֶר אֱלֹהִ֔ים </span>|<span class="hb">יֹּ֣אמֶר </span>|<span class="hb">יֹּ֣אמֶר </span>|<span class="hb">אֱלֹהִ֔ים </span>|<span class="hb">אֱלֹהִ֔ים </span>
7|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=7&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:7" sec="Genesis 1:7">Genesis 1:7</a>וַיַּ֣עַשׂ אֱלֹהִים֮ אֶת־הָרָקִיעַ֒ וַיַּבְדֵּ֗ל בֵּ֤ין הַמַּ֨יִם֙ אֲשֶׁר֙ מִתַּ֣חַת לָרָקִ֔יעַ וּבֵ֣ין הַמַּ֔יִם אֲשֶׁ֖ר מֵעַ֣ל לָרָקִ֑יעַ וַֽיְהִי־כֵֽן׃ </span>|<span class="hb">וַיַּ֣עַשׂ אֱלֹהִים֮ אֶת־הָרָקִיעַ֒ </span>|<span class="hb">יַּ֣עַשׂ </span>|<span class="hb">יַּ֣עַשׂ </span>|<span class="hb">אֱלֹהִים֮ </span>|<span class="hb">אֱלֹהִים֮ </span>
8|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=8&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:8" sec="Genesis 1:8">Genesis 1:8</a>וַיִּקְרָ֧א אֱלֹהִ֛ים לָֽרָקִ֖יעַ שָׁמָ֑יִם וַֽיְהִי־עֶ֥רֶב וַֽיְהִי־בֹ֖קֶר יֹ֥ום שֵׁנִֽי׃ פ </span>|<span class="hb">וַיִּקְרָ֧א אֱלֹהִ֛ים לָֽרָקִ֖יעַ שָׁמָ֑יִם </span>|<span class="hb">יִּקְרָ֧א </span>|<span class="hb">יִּקְרָ֧א </span>|<span class="hb">אֱלֹהִ֛ים </span>|<span class="hb">אֱלֹהִ֛ים </span>
9|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=9&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:9" sec="Genesis 1:9">Genesis 1:9</a>וַיֹּ֣אמֶר אֱלֹהִ֗ים יִקָּו֨וּ הַמַּ֜יִם מִתַּ֤חַת הַשָּׁמַ֨יִם֙ אֶל־מָקֹ֣ום אֶחָ֔ד וְתֵרָאֶ֖ה הַיַּבָּשָׁ֑ה וַֽיְהִי־כֵֽן׃ </span>|<span class="hb">וַיֹּ֣אמֶר אֱלֹהִ֗ים </span>|<span class="hb">יֹּ֣אמֶר </span>|<span class="hb">יֹּ֣אמֶר </span>|<span class="hb">אֱלֹהִ֗ים </span>|<span class="hb">אֱלֹהִ֗ים </span>
10|<span class="hb"><a class="vn" href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=10&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:10" sec="Genesis 1:10">Genesis 1:10</a>וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לַיַּבָּשָׁה֙ אֶ֔רֶץ וּלְמִקְוֵ֥ה הַמַּ֖יִם קָרָ֣א יַמִּ֑ים וַיַּ֥רְא אֱלֹהִ֖ים כִּי־טֹֽוב׃ </span>|<span class="hb">וַיִּקְרָ֨א אֱלֹהִ֤ים׀ לַיַּבָּשָׁה֙ אֶ֔רֶץ </span>|<span class="hb">יִּקְרָ֨א </span>|<span class="hb">יִּקְרָ֨א </span>|<span class="hb">אֱלֹהִ֤ים׀ </span>|<span class="hb">אֱלֹהִ֤ים׀ </span>

Now we want to restrict results to ordinary chapters:

In [16]:
query2 = '''
ochapter
  verse
    clause
      phrase function=Pred
        word pdp=verb nu=sg
      phrase function=Subj
        =: word pdp=subs nu=pl
        :=
'''

Note that we use the name of a set here: `ochapter`. 
It is not a known node type in the BHSA, so we have to tell it what it means.
We do that by passing a dictionary of custom sets.
The keys are the names of the sets, which are the values.

Then we may use those keys in queries, everywhere where a node type is expected.

In [17]:
results2 = A.search(query2, sets=sets)

  1.57s 7 results


In [18]:
A.table(results2)

n | chapter | verse | clause | phrase | word | phrase | word
--- | --- | --- | --- | --- | --- | --- | ---
1|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=47&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 47" sec="Psalms 47">Psalms 47</a></span>|<span class="hb"><span class="vn">Psalms 47:6</span>עָלָ֣ה אֱ֭לֹהִים בִּתְרוּעָ֑ה יְ֝הֹוָ֗ה בְּקֹ֣ול שֹׁופָֽר׃ </span>|<span class="hb">עָלָ֣ה אֱ֭לֹהִים בִּתְרוּעָ֑ה </span>|<span class="hb">עָלָ֣ה </span>|<span class="hb">עָלָ֣ה </span>|<span class="hb">אֱ֭לֹהִים </span>|<span class="hb">אֱ֭לֹהִים </span>
2|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=47&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 47" sec="Psalms 47">Psalms 47</a></span>|<span class="hb"><span class="vn">Psalms 47:9</span>מָלַ֣ךְ אֱ֭לֹהִים עַל־גֹּויִ֑ם אֱ֝לֹהִ֗ים יָשַׁ֤ב׀ עַל־כִּסֵּ֬א קָדְשֹֽׁו׃ </span>|<span class="hb">מָלַ֣ךְ אֱ֭לֹהִים עַל־גֹּויִ֑ם </span>|<span class="hb">מָלַ֣ךְ </span>|<span class="hb">מָלַ֣ךְ </span>|<span class="hb">אֱ֭לֹהִים </span>|<span class="hb">אֱ֭לֹהִים </span>
3|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=47&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 47" sec="Psalms 47">Psalms 47</a></span>|<span class="hb"><span class="vn">Psalms 47:9</span>מָלַ֣ךְ אֱ֭לֹהִים עַל־גֹּויִ֑ם אֱ֝לֹהִ֗ים יָשַׁ֤ב׀ עַל־כִּסֵּ֬א קָדְשֹֽׁו׃ </span>|<span class="hb">אֱ֝לֹהִ֗ים יָשַׁ֤ב׀ עַל־כִּסֵּ֬א קָדְשֹֽׁו׃ </span>|<span class="hb">יָשַׁ֤ב׀ </span>|<span class="hb">יָשַׁ֤ב׀ </span>|<span class="hb">אֱ֝לֹהִ֗ים </span>|<span class="hb">אֱ֝לֹהִ֗ים </span>
4|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=53&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 53" sec="Psalms 53">Psalms 53</a></span>|<span class="hb"><span class="vn">Psalms 53:3</span>אֱֽלֹהִ֗ים מִשָּׁמַיִם֮ הִשְׁקִ֪יף עַֽל־בְּנֵ֫י אָדָ֥ם לִ֭רְאֹות הֲיֵ֣שׁ מַשְׂכִּ֑יל דֹּ֝רֵ֗שׁ אֶת־אֱלֹהִֽים׃ </span>|<span class="hb">אֱֽלֹהִ֗ים מִשָּׁמַיִם֮ הִשְׁקִ֪יף עַֽל־בְּנֵ֫י אָדָ֥ם </span>|<span class="hb">הִשְׁקִ֪יף </span>|<span class="hb">הִשְׁקִ֪יף </span>|<span class="hb">אֱֽלֹהִ֗ים </span>|<span class="hb">אֱֽלֹהִ֗ים </span>
5|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=53&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 53" sec="Psalms 53">Psalms 53</a></span>|<span class="hb"><span class="vn">Psalms 53:6</span>שָׁ֤ם׀ פָּֽחֲדוּ־פַחַד֮ לֹא־הָ֪יָה֫ פָ֥חַד כִּֽי־אֱלֹהִ֗ים פִּ֭זַּר עַצְמֹ֣ות חֹנָ֑ךְ הֱ֝בִשֹׁ֗תָה כִּֽי־אֱלֹהִ֥ים מְאָסָֽם׃ </span>|<span class="hb">כִּֽי־אֱלֹהִ֗ים פִּ֭זַּר עַצְמֹ֣ות חֹנָ֑ךְ </span>|<span class="hb">פִּ֭זַּר </span>|<span class="hb">פִּ֭זַּר </span>|<span class="hb">אֱלֹהִ֗ים </span>|<span class="hb">אֱלֹהִ֗ים </span>
6|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Psalmi&chapter=70&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Psalms 70" sec="Psalms 70">Psalms 70</a></span>|<span class="hb"><span class="vn">Psalms 70:5</span>יָ֘שִׂ֤ישׂוּ וְיִשְׂמְח֨וּ׀ בְּךָ֗ כָּֽל־מְבַ֫קְשֶׁ֥יךָ וְיֹאמְר֣וּ תָ֭מִיד יִגְדַּ֣ל אֱלֹהִ֑ים אֹ֝הֲבֵ֗י יְשׁוּעָתֶֽךָ׃ </span>|<span class="hb">יִגְדַּ֣ל אֱלֹהִ֑ים </span>|<span class="hb">יִגְדַּ֣ל </span>|<span class="hb">יִגְדַּ֣ל </span>|<span class="hb">אֱלֹהִ֑ים </span>|<span class="hb">אֱלֹהִ֑ים </span>
7|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Chronica_II&chapter=27&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="2_Chronicles 27" sec="2_Chronicles 27">2_Chronicles 27</a></span>|<span class="hb"><span class="vn">2_Chronicles 27:6</span>וַיִּתְחַזֵּ֖ק יֹותָ֑ם כִּ֚י הֵכִ֣ין דְּרָכָ֔יו לִפְנֵ֖י יְהוָ֥ה אֱלֹהָֽיו׃ </span>|<span class="hb">כִּ֚י הֵכִ֣ין דְּרָכָ֔יו לִפְנֵ֖י יְהוָ֥ה אֱלֹהָֽיו׃ </span>|<span class="hb">הֵכִ֣ין </span>|<span class="hb">הֵכִ֣ין </span>|<span class="hb">דְּרָכָ֔יו </span>|<span class="hb">דְּרָכָ֔יו </span>

## Custom sets in the browser

We save the sets in a file.
But before we do so, we also want to save all ordinary verses in a set, and all ordinary words. 

In [19]:
queryV = f'''
verse
/without/
  word freq_lex<{FREQ}
/-/
'''
resultsV = A.search(queryV, shallow=True)
sets['overse'] = resultsV

  0.47s 2757 results


In [20]:
sets['oword'] = {w for w in F.otype.s('word') if F.freq_lex.v(w) >= FREQ}

In [21]:
SETS_FILE = os.path.expanduser('~/Downloads/ordinary.set')
writeSets(sets, SETS_FILE)

True

In [22]:
testSets = readSets(SETS_FILE)
for s in sorted(testSets):
    elems = len(testSets[s])
    oelems = len(sets[s])
    print(f'{s} with {elems} nb {elems - oelems}')

ochapter with 60 nb 0
overse with 2757 nb 0
oword with 361485 nb 0


Now you can start your TF browser as follows:

```sh
text-fabric bhsa --sets=~/Downloads/ordinary.set
```

and then you can run the same queries over there!

# Appendix: investigation

Let's investigate the number of ordinary chapters with shifting definitions of ordinary

In [23]:
allChapters = F.otype.s('chapter')
longestChapter = max(len(L.d(chapter, otype='word')) for chapter in allChapters)

print(f'There are {len(allChapters)} chapters, the longest is {longestChapter} words')

There are 929 chapters, the longest is 1603 words


In [24]:
def getOrdinary(freq, amount):
    results = []

    for chapter in allChapters:
        if len([
            word for word in L.d(chapter, otype='word') if F.freq_lex.v(word) < freq
        ]) < amount:
            results.append(chapter)
    return results

In [27]:
def overview(freq):
    for amount in range(20, 1700, 50):
        results = getOrdinary(freq, amount)
        print(f'for freq={freq:>3} and amount={amount:>4}: {len(results):>4} ordinary chapters')
        if len(results) >= len(allChapters):
            break

In [28]:
for freq in (40, 70, 100):
    overview(freq)

for freq= 40 and amount=  20:  140 ordinary chapters
for freq= 40 and amount=  70:  758 ordinary chapters
for freq= 40 and amount= 120:  885 ordinary chapters
for freq= 40 and amount= 170:  908 ordinary chapters
for freq= 40 and amount= 220:  919 ordinary chapters
for freq= 40 and amount= 270:  923 ordinary chapters
for freq= 40 and amount= 320:  924 ordinary chapters
for freq= 40 and amount= 370:  925 ordinary chapters
for freq= 40 and amount= 420:  926 ordinary chapters
for freq= 40 and amount= 470:  928 ordinary chapters
for freq= 40 and amount= 520:  929 ordinary chapters
for freq= 70 and amount=  20:   60 ordinary chapters
for freq= 70 and amount=  70:  551 ordinary chapters
for freq= 70 and amount= 120:  842 ordinary chapters
for freq= 70 and amount= 170:  890 ordinary chapters
for freq= 70 and amount= 220:  915 ordinary chapters
for freq= 70 and amount= 270:  922 ordinary chapters
for freq= 70 and amount= 320:  923 ordinary chapters
for freq= 70 and amount= 370:  923 ordinary ch

# Next

You have seen how to mingle sets with queries.

Time to enter the race for space:
[relations](searchRelations.ipynb)

---

[basic](search.ipynb)
[advanced](searchAdvanced.ipynb)
sets
[relations](searchRelations.ipynb)
[quantifiers](searchQuantifiers.ipynb)
[rough](searchRough.ipynb)
[gaps](searchGaps.ipynb)